In [60]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

import os
import numpy as np
import sys
import pandas as pd
sys.path.append("/usr/local/lib/python3.5/dist-packages/")
import nibabel as nib

import matplotlib as mpl
import matplotlib.pyplot as plt

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim



df = np.genfromtxt("others/MRIQualityControl/preprocessing/encoded_track2_labels.csv",delimiter=',')
x_loc = df[1:,2]
y_loc = df[1:,3]



class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        
        self.conv1 = nn.Conv3d(1, 6, 8, stride=2)
        self.conv2 = nn.Conv3d(6,16,3)
        
        
        #self.conv1 = nn.Conv2d(1, 6, 3)
        #self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 21 * 25 * 21, 5)  # 6*6 from image dimension
        self.fc2 = nn.Linear(5, 20)
        self.fc3 = nn.Linear(20, 1)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool3d(F.relu(self.conv1(x)), (2, 2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool3d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = torch.sigmoid(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

optimizer = optim.SGD(net.parameters(), lr=0.005)
criterion = nn.BCELoss()



batch_size = 20
D, R, C = (182, 218,182)
x = torch.zeros((batch_size,1,D,R,C))
y = torch.zeros(batch_size)
offset = 0


Net(
  (conv1): Conv3d(1, 6, kernel_size=(8, 8, 8), stride=(2, 2, 2))
  (conv2): Conv3d(6, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (fc1): Linear(in_features=176400, out_features=5, bias=True)
  (fc2): Linear(in_features=5, out_features=20, bias=True)
  (fc3): Linear(in_features=20, out_features=1, bias=True)
)


In [61]:

# in your training loop:
for i in range(100):
    for j in range(batch_size):
        loc = x_loc[j+offset]
        example_filename = "../dataset/Sub-" + str(int(loc)) + ".nii.gz"
        img = nib.load(example_filename)
        data = img.get_fdata()
        x[j,0,:,:,:] = torch.from_numpy(data).float()
        y[j] = y_loc[j+offset]
    offset = j+offset
    input = x/x.max()
    optimizer.zero_grad()   # zero the gradient buffers
    output = net(input)
    loss = criterion(output.view(batch_size), y.view(batch_size))
    loss.backward()
    optimizer.step()    # Does the update
    print(loss)
    print(output)
    print(y)
    print(input.sum())

RuntimeError: [enforce fail at CPUAllocator.cpp:64] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 33622589440 bytes. Error code 12 (Cannot allocate memory)


In [51]:
offset = 0

tensor(2336306.5000)